In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from sklearn.naive_bayes import MultinomialNB
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
import gensim.downloader as api

In [12]:
df = pd.read_csv('/content/ad_click_dataset.csv')

# Display column names and dataset size
print("Column Names:", df.columns)
print("Dataset Size:", df.shape)

Column Names: Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
Dataset Size: (10000, 9)


In [13]:
# 2) Work out the gaps (if possible, fill them in or delete them)
# Check for missing values
print("Missing Values:", df.isnull().sum())

# Fill missing values (numerical) with mean, categorical with mode
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns

imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

df[num_cols] = imputer_num.fit_transform(df[num_cols])
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

# Confirm no more missing values
print("Missing Values After Imputation:", df.isnull().sum())
print(df)

Missing Values: id                     0
full_name              0
age                 4766
gender              4693
device_type         2000
ad_position         2000
browsing_history    4782
time_of_day         2000
click                  0
dtype: int64
Missing Values After Imputation: id                  0
full_name           0
age                 0
gender              0
device_type         0
ad_position         0
browsing_history    0
time_of_day         0
click               0
dtype: int64
          id full_name        age      gender device_type ad_position  \
0      670.0   User670  22.000000      Female     Desktop         Top   
1     3044.0  User3044  40.197363        Male     Desktop         Top   
2     5912.0  User5912  41.000000  Non-Binary     Desktop        Side   
3     5418.0  User5418  34.000000        Male     Desktop      Bottom   
4     9452.0  User9452  39.000000  Non-Binary     Desktop      Bottom   
...      ...       ...        ...         ...         ...       

In [14]:
df = df.drop(columns=['id', 'full_name'], errors='ignore')
df = pd.get_dummies(df, columns=['gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']).astype(int)
df

,age,click,gender_Female,gender_Male,gender_Non-Binary,device_type_Desktop,device_type_Mobile,device_type_Tablet,ad_position_Bottom,ad_position_Side,ad_position_Top,browsing_history_Education,browsing_history_Entertainment,browsing_history_News,browsing_history_Shopping,browsing_history_Social Media,time_of_day_Afternoon,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
0,22,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0
1,40,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0
2,41,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1
3,34,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0
4,39,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
9996,40,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0
9997,40,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0
9998,40,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0


In [15]:
# 4) Normalize the data
X = df.drop(columns=['click'])
y = df['click']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [19]:
# Convert the labels from pandas Series to NumPy arrays
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

# Proceed with the rest of the code as before
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Creating a fully connected neural network
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Initializing the model, optimizer, and loss function
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(set(y_train))
model = FullyConnectedNN(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 20
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch [2/20], Loss: 0.6729
Epoch [4/20], Loss: 0.6644
Epoch [6/20], Loss: 0.6585
Epoch [8/20], Loss: 0.6541
Epoch [10/20], Loss: 0.6504
Epoch [12/20], Loss: 0.6472
Epoch [14/20], Loss: 0.6447
Epoch [16/20], Loss: 0.6429
Epoch [18/20], Loss: 0.6417
Epoch [20/20], Loss: 0.6409
Accuracy: 64.70%


In [35]:
# Define transforms for your custom dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize all images to the same size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize RGB images
])

# Load all data from your train folder
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/train', transform=transform)

# Split the dataset into 80% training and 20% testing
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # Remaining 20% for testing
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for training and testing
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Define the CNN architecture for your 3-class problem
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)  # 3 input channels for RGB images
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)  # Adjust the size based on your input image size
        self.fc2 = nn.Linear(128, 3)  # 3 output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

Epoch [1/5], Loss: 0.0009
Epoch [2/5], Loss: 0.0000
Epoch [3/5], Loss: 0.0000
Epoch [4/5], Loss: 0.0000
Epoch [5/5], Loss: 0.0000
Accuracy: 100.00%


In [31]:
# Define transforms for your custom dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # ResNet expects this normalization
])

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer to match the number of classes in your dataset (3 classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 classes

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer (only the final layer's parameters are optimized)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Testing the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Epoch [1/10], Loss: 0.7383
Epoch [2/10], Loss: 0.3315
Epoch [3/10], Loss: 0.2248
Epoch [4/10], Loss: 0.1585
Epoch [5/10], Loss: 0.1331
Epoch [6/10], Loss: 0.1047
Epoch [7/10], Loss: 0.1131
Epoch [8/10], Loss: 0.0805
Epoch [9/10], Loss: 0.0920
Epoch [10/10], Loss: 0.0664
Accuracy: 99.40%


In [41]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the data from CSV files
reviews = pd.read_csv('reviews.csv')
labels = pd.read_csv('labels.csv')

# Merge the two datasets based on 'id' column
data = pd.merge(reviews, labels, on='id')
data = data.drop(columns=['id'], errors='ignore')

# Data Cleaning Function
def preprocess_text(text):
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Convert to lower case
    text = text.lower()

    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

# Preprocess text and create dataset
data['cleaned_text'] = data['text'].apply(preprocess_text)

# Convert sentiment labels to binary
data['sentiment'] = data['sentiment'].map({'Positive': 1, 'Negative': 0})

# TextDataset class for creating dataset from the text and labels
class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab=None, max_length=100):
        self.texts = texts
        self.labels = labels.astype(int)  # Ensure labels are integer type
        self.max_length = max_length

        # Build vocabulary
        if vocab is None:
            all_words = [word for text in texts for word in word_tokenize(text.lower())]
            self.vocab = {word: i+1 for i, (word, _) in enumerate(Counter(all_words).items())}
            self.vocab['<PAD>'] = 0
        else:
            self.vocab = vocab

        self.vocab_size = len(self.vocab)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and convert to indices
        tokens = word_tokenize(self.texts[idx].lower())
        indexed = [self.vocab.get(word, 0) for word in tokens]

        # Pad sequence to max_length
        if len(indexed) > self.max_length:
            indexed = indexed[:self.max_length]
        else:
            indexed = indexed + [self.vocab['<PAD>']] * (self.max_length - len(indexed))

        # Return the indexed tokens and the corresponding label as a tensor
        return torch.tensor(indexed), torch.tensor(self.labels[idx])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['sentiment'], test_size=0.2, random_state=42)

# Create dataset for train and test
train_dataset = TextDataset(X_train.values, y_train.values)
test_dataset = TextDataset(X_test.values, y_test.values, vocab=train_dataset.vocab)

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.embedding(x)
        h_0 = torch.zeros(1, x.size(0), hidden_dim).to(x.device)  # Initialize hidden state
        out, _ = self.rnn(x, h_0)  # RNN layer
        out = self.fc(out[:, -1, :])  # Only use the last output of RNN for classification
        return out

# Initialize the model
embedding_dim = 100
hidden_dim = 128
output_size = 2  # Binary classification (Positive, Negative)
vocab_size = len(train_dataset.vocab)

model = RNNModel(vocab_size, embedding_dim, hidden_dim, output_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (texts, labels) in enumerate(train_loader):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}')

# Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/10, Loss: 0.6887
Epoch 2/10, Loss: 0.6810
Epoch 3/10, Loss: 0.6791
Epoch 4/10, Loss: 0.6800
Epoch 5/10, Loss: 0.6791
Epoch 6/10, Loss: 0.6856
Epoch 7/10, Loss: 0.6705
Epoch 8/10, Loss: 0.6825
Epoch 9/10, Loss: 0.6787
Epoch 10/10, Loss: 0.6718
Test Accuracy: 68.63%


In [44]:
# Load GloVe pre-trained embeddings (50d as an example)
glove_vectors = api.load("glove-wiki-gigaword-50")

# Initialize embedding matrix
embedding_dim = 50
vocab_size = len(train_dataset.vocab)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Build embedding matrix using pre-trained GloVe vectors
for word, idx in train_dataset.vocab.items():
    if word in glove_vectors:
        embedding_matrix[idx] = glove_vectors[word]
    else:
        # Random initialization for words not in the GloVe vocabulary
        embedding_matrix[idx] = np.random.normal(0, 1, embedding_dim)

# Define the model, this time using pre-trained embeddings
class RNNModelPretrained(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size, embedding_matrix):
        super(RNNModelPretrained, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.embedding(x)
        h_0 = torch.zeros(1, x.size(0), hidden_dim).to(x.device)
        out, _ = self.rnn(x, h_0)
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model using pre-trained embeddings
hidden_dim = 128
output_size = 2  # Binary classification (Positive, Negative)

model_pretrained = RNNModelPretrained(vocab_size, embedding_dim, hidden_dim, output_size, embedding_matrix)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_pretrained = model_pretrained.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pretrained.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_pretrained.train()
    total_loss = 0
    for batch_idx, (texts, labels) in enumerate(train_loader):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_pretrained(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}')

# Evaluate on test set
model_pretrained.eval()
correct = 0
total = 0
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model_pretrained(texts)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

[==================================================] 100.0% 66.0/66.0MB downloaded
Epoch 1/10, Loss: 0.6968
Epoch 2/10, Loss: 0.7295
Epoch 3/10, Loss: 0.7105
Epoch 4/10, Loss: 0.6869
Epoch 5/10, Loss: 0.6938
Epoch 6/10, Loss: 0.6765
Epoch 7/10, Loss: 0.6824
Epoch 8/10, Loss: 0.6881
Epoch 9/10, Loss: 0.6839
Epoch 10/10, Loss: 0.6874
Test Accuracy: 68.63%
